In [2]:
import os
import sys
import warnings
import pandas as pd
from tqdm.notebook import tqdm

base_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(f"base_path: {base_path}")
sys.path.append(base_path)

base_path: /home/toc3/sr-press


In [3]:
import express.config as config
from express.databases import SQLiteDatabase
from express.datasets import PressingDataset
from express.utils import add_names

In [4]:
TRAIN_DB_PATH = os.path.join(base_path, "stores/train_database.sqlite")
TEST_DB_PATH = os.path.join(base_path, "stores/test_database.sqlite")

train_db = SQLiteDatabase(TRAIN_DB_PATH)
test_db = SQLiteDatabase(TEST_DB_PATH)

print("train_db:", train_db)
print("test_db:", test_db)

train_db: <express.databases.sqlite.SQLiteDatabase object at 0x7f1c594c2c10>
test_db: <express.databases.sqlite.SQLiteDatabase object at 0x7f1c59528e80>


In [5]:
game_id = 3895302

df_actions = add_names(train_db.actions(game_id))
df_actions

original_event_id  period_id  \
game_id action_id                                                    
3895302 0          221b0c8d-6386-4ae8-bb4a-a1dc98742312          1   
        1          77809242-1460-4395-8779-94a0cfc275b1          1   
        2          ff56e821-21e9-4cef-ba2a-7eb5eb3769c6          1   
        3          cf9088bc-7e59-4d57-8ac5-31658da858bb          1   
        4          4464cb75-f45f-4508-8444-2560d1625d06          1   
...                                                 ...        ...   
        3025       4e61f75d-564b-4478-a2f0-4b7e20e8600e          2   
        3026                                       None          2   
        3027       91466764-aec7-4fac-a024-dd3deee3c737          2   
        3028       e75b1a76-8aca-461f-846f-70b8b9e35fa9          2   
        3029       dcc0c591-7ebd-4e42-8206-04bfefa94f33          2   

                   time_seconds  team_id  player_id  start_x  start_y  \
game_id action_id                                                       
3895302 0                3.4170      176      34870  52.0625   33.660   
        1                3.8700      176      12299  53.8125   34.340   
        2                4.7320      176      12299  53.8125   34.085   
        3                6.7280      176      31100  74.7250   36.295   
        4                7.6220      176      31100  74.7250   36.720   
...                         ...      ...        ...      ...      ...   
        3025          2661.1130      904      10336  57.4000   45.220   
        3026          2661.7455      176      24179  55.1250   44.795   
        3027          2662.5800      904      40724  77.7875   21.335   
        3028          2662.8440      176      24179  84.0875   19.890   
        3029          2665.0790      904      40724  92.6625   26.945   

                      end_x    end_y  bodypart_id  ...  possession_team_id  \
game_id action_id                                  ...                       
3895302 0           53.8125  34.3400            5  ...               176.0   
        1           53.8125  34.0850            0  ...               176.0   
        2           74.7250  36.2950            5  ...               176.0   
        3           74.7250  36.7200            0  ...               176.0   
        4           76.3875  53.8050            5  ...               176.0   
...                     ...      ...          ...  ...                 ...   
        3025        77.7875  21.3350            4  ...               904.0   
        3026        84.0875  19.8900            0  ...                 NaN   
        3027        92.6625  26.9450            0  ...               904.0   
        3028        84.0875  19.8900            0  ...               904.0   
        3029       104.5625  35.9975            5  ...               904.0   

                   play_pattern_name  under_pressure  \
game_id action_id                                      
3895302 0              From Kick Off               0   
        1              From Kick Off               0   
        2              From Kick Off               0   
        3              From Kick Off               0   
        4              From Kick Off               0   
...                              ...             ...   
        3025            From Counter               0   
        3026                    None               0   
        3027            From Counter               1   
        3028            From Counter               0   
        3029            From Counter               0   

                                                               extra  \
game_id action_id                                                      
3895302 0          {'pass': {'recipient': {'id': 12299, 'name': '...   
        1                  {'carry': {'end_location': [59.0, 40.6]}}   
        2          {'pass': {'recipient': {'id': 31100, 'name': '...   
        3                  {'carry': {'end_location': [35.1, 43.7]}}   
 

In [6]:
from express import features as fs
from express import labels as ls

all_features = [f.__name__ for f in fs.all_features]
all_labels = [f.__name__ for f in ls.all_labels]
print("Features:", all_features)
print("Labels:", all_labels)

Features: ['actiontype', 'actiontype_onehot', 'result', 'result_onehot', 'bodypart', 'bodypart_onehot', 'time', 'startlocation', 'relative_startlocation', 'endlocation', 'relative_endlocation', 'startpolar', 'endpolar', 'movement', 'team', 'time_delta', 'space_delta', 'goalscore', 'angle', 'under_pressure', 'packing_rate', 'ball_height_onehot', 'player_possession_time', 'speed', 'nb_opp_in_path', 'dist_opponent', 'defenders_in_3m_radius', 'closest_3_players']
Labels: ['concede_shots', 'counterpress']


In [7]:
train_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "datasets", "train"),
    xfns=["actiontype"],#["actiontype", "actiontype_onehot", "result", "result_onehot", "bodypart", "bodypart_onehot"],#["startlocation", "time", "goalscore", "defenders_in_3m_radius", "dist_opponent", "closest_3_players"], #all_features
    yfns=["counterpress"], #all_labels
    load_cached =False,
    nb_prev_actions = 3,
)

test_dataset = PressingDataset(
    path= os.path.join(base_path, "stores", "datasets", "test"),
    xfns=["actiontype"],#["startlocation", "time", "goalscore", "defenders_in_3m_radius", "dist_opponent", "closest_3_players"], #all_features
    yfns=["counterpress"], #all_labels
    load_cached =False,
    nb_prev_actions = 3
)

In [8]:
train_dataset.create(train_db)
test_dataset.create(test_db)

Output()

Output()

Output()

Output()

In [9]:
train_dataset.features

type_id_a0  type_id_a1  type_id_a2
game_id action_id                                    
3788741 3                  23         0.0        21.0
        19                 23         0.0        21.0
        27                 23         0.0         2.0
        31                 23        21.0        19.0
        35                 23        21.0        21.0
...                       ...         ...         ...
3943043 2283               23         2.0        22.0
        2286               23         2.0         0.0
        2302               23         4.0        21.0
        2304               23        21.0        23.0
        2309               23        21.0        18.0

[36948 rows x 3 columns]

In [10]:
test_dataset.features

type_id_a0  type_id_a1  type_id_a2
game_id action_id                                    
3857254 4                  23        21.0         0.0
        14                 23        21.0        18.0
        17                 23        21.0         0.0
        22                 23        21.0        21.0
        34                 23        21.0         0.0
...                       ...         ...         ...
3869685 3013               23        21.0         0.0
        3020               23         0.0        21.0
        3031               23        21.0        19.0
        3041               23        21.0         7.0
        3045               23        21.0        21.0

[14504 rows x 3 columns]

In [11]:
train_dataset.labels

counterpress
game_id action_id              
3788741 3                 False
        19                False
        27                False
        31                 True
        35                 True
...                         ...
3943043 2283              False
        2286              False
        2302              False
        2304              False
        2309              False

[36948 rows x 1 columns]

In [12]:
test_dataset.labels

counterpress
game_id action_id              
3857254 4                 False
        14                False
        17                False
        22                False
        34                False
...                         ...
3869685 3013               True
        3020              False
        3031              False
        3041              False
        3045              False

[14504 rows x 1 columns]

In [13]:
train_dataset.labels["counterpress"].value_counts()

False    30044
True      6904
Name: counterpress, dtype: int64

In [14]:
test_dataset.labels["counterpress"].value_counts()

False    11515
True      2989
Name: counterpress, dtype: int64